In [33]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


Tucker Decomposition (d = 3) case (HOSVD)

step 1: unfold the tensor

given a rank 3 tensor, we can think of it taking a shape of a cube, being characterized by depth/height/width (labed as I,J,K paramters.) Giving us (IxJxK) tensor components. We aim to flatten the matrix by finding 3 matrices, which each holds information of the tensor's rows, columns and depths. 

Thus we consider 3 different tensor slicing to collect the rows, columns and depths to give us the T1, T2, T3 matrices.

In [23]:
#randomized rank 3 tensor, with i,j,k randomly assigned between 1-5, and tensor entires between 1-50
def rand_tensor():
    i = np.random.randint(1, 6)
    j = np.random.randint(1, 6)
    k = np.random.randint(1, 6)
    
    tensor = np.random.randint(0, 51, size=(i, j, k))

    return tensor

In [203]:
def flatten(tensor):
    i,j,k = np.shape(tensor) #returns tuple (i,j,k)

    T1 = tensor.transpose(0, 2, 1).reshape(i, -1)
    T2 = tensor.transpose(1, 2, 0).reshape(j, -1)
    T3 = tensor.transpose(2, 1, 0).reshape(k, -1)

    #tensor transpose rotates the tensor into its different sides, and 
    #reshape reduces the dimension of the tensor, which in this case flattens it into a matrix.
    return T1, T2, T3

Here we have successfully collected all the rows, columns and depths vectors.

Step 2:
Now that we have the unfolded matrices, we can perform SVD on them to obtain U1, U2, and U3 (U part of the SVD, A = U sigma VTranspose)

In [26]:
def factor_matrices(T1, T2, T3):
    U1, S1, Vt1 = np.linalg.svd(T1, full_matrices=False)
    U2, S2, Vt2 = np.linalg.svd(T2, full_matrices=False)
    U3, S3, Vt3 = np.linalg.svd(T3, full_matrices=False) 
    #we keep singular values here, so that we could ignore some of these entires to simplify calculation when we have larger matrices
    return U1, U2, U3

In [31]:
def core_tensor(tensor, U1, U2, U3):
    X_step1 = np.tensordot(tensor, U1, axes=(0, 0))
    X_step2 = np.tensordot(X_step1, U2, axes=(0, 0))
    X = np.tensordot(X_step2, U3, axes=(0, 0))
    return X

In [49]:
def reconstruct(X, U1, U2, U3):
    A_step1 = np.tensordot(X, U1, axes=(0, 1))
    A_step2 = np.tensordot(A_step1, U2, axes=(0, 1))
    A = np.tensordot(A_step2, U3, axes=(0, 1))
    return A

Example:

In [36]:
tensor = rand_tensor()
print(tensor)

[[[ 5 24]
  [46 43]
  [27 22]
  [28  3]]

 [[ 3 35]
  [34 44]
  [29 19]
  [38 11]]

 [[ 0 41]
  [36 30]
  [18 41]
  [18 23]]]


In [43]:
T1, T2, T3 = flatten(tensor)
print(T1)
print(T2)
print(T3)

[[ 5 46 27 28 24 43 22  3]
 [ 3 34 29 38 35 44 19 11]
 [ 0 36 18 18 41 30 41 23]]
[[ 5 24  3 35  0 41]
 [46 43 34 44 36 30]
 [27 22 29 19 18 41]
 [28  3 38 11 18 23]]
[[ 5  3  0 46 34 36 27 29 18 28 38 18]
 [24 35 41 43 44 30 22 19 41  3 11 23]]


In [50]:
U1, U2, U3 = factor_matrices(T1, T2, T3)
X = core_tensor(tensor, U1, U2, U3)

A = reconstruct(X,U1,U2,U3)
print(A)

[[[ 5.00000000e+00  2.40000000e+01]
  [ 4.60000000e+01  4.30000000e+01]
  [ 2.70000000e+01  2.20000000e+01]
  [ 2.80000000e+01  3.00000000e+00]]

 [[ 3.00000000e+00  3.50000000e+01]
  [ 3.40000000e+01  4.40000000e+01]
  [ 2.90000000e+01  1.90000000e+01]
  [ 3.80000000e+01  1.10000000e+01]]

 [[-5.58626961e-15  4.10000000e+01]
  [ 3.60000000e+01  3.00000000e+01]
  [ 1.80000000e+01  4.10000000e+01]
  [ 1.80000000e+01  2.30000000e+01]]]


Combine into the final HOSVD function:

In [51]:
def HOSVD(A):
    T1, T2, T3 = flatten(A)
    U1, U2, U3 = factor_matrices(T1, T2, T3)
    X = core_tensor(tensor, U1, U2, U3)
    A = reconstruct(X,U1,U2,U3)
    return A

Note that this method can be generalized for rank d tensors, by having T1,T2,...,Td and following the same steps.

references for this part:
https://en.wikipedia.org/wiki/Higher-order_singular_value_decomposition
https://www.math.ucdavis.edu/~saito/data/tensor/lathauwer-etal_mulilinear-SVD.pdf (good diagram)
https://www.math.ucdavis.edu/~saito/data/tensor/bergqvist-larsson-hosvd.pdf (good visuals/short poster)

extra reference along-side wiki on HOSVD: https://www.math.ucdavis.edu/~saito/data/tensor/kolda-bader_tensor-decomp-siamrev.pdf

CP Decomposition:

In [25]:
def khatrirao(A,B):
    num_cols = A.shape[1]
    columns = [np.kron(A[:, i], B[:, i]) for i in range(num_cols)]
    return np.column_stack(columns)

In [26]:
def hadamard(A,B):
    return A * B

In [27]:
def optimize_a(T1,B,C):
    hold = hadamard(C.T@C, B.T@B)
    A = T1 @ khatrirao(C,B)
    A = A @ np.linalg.pinv(hold)
    return A

In [28]:
def optimize_b(T2,C,A):
    hold = hadamard(C.T@C, A.T@A)
    B = T2 @ khatrirao(C,A)
    B = B @ np.linalg.pinv(hold)
    return B

In [29]:
def optimize_c(T3, B, A):
    hold = hadamard(B.T@B, A.T@A)
    C = T3 @ khatrirao(B,A)
    C = C @ np.linalg.pinv(hold)
    return C

In [81]:
def CP(tensor, rank, max_iter, tol=1e-4):
    i,j,k = np.shape(tensor)
    T1, T2, T3 = flatten(tensor)
    A = np.random.random((i, rank))
    B = np.random.random((j, rank))
    C = np.random.random((k, rank))

    for iteration in range(max_iter):
        #A_old = A.copy()
        
        A = optimize_A(T1, B, C)
        B = optimize_b(T2, C, A)
        C = optimize_c(T3, B, A)

        #change = np.linalg.norm(A - A_old) (change per iteration termination condition, will set this up later)
        #if change < tol:
        #    break
    return A,B,C

In [80]:
def reconstruct_CP(A, B, C):
    I, R = A.shape
    J, _ = B.shape
    K, _ = C.shape
    
    tensor = np.zeros((I, J, K))
    
    for i in range(R):
        a_col = A[:, i]
        b_col = B[:, i]
        c_col = C[:, i]
        
        rank1_component = a_col[:, None, None] * b_col[None, :, None] * c_col[None, None, :]
        tensor += rank1_component
        
    return tensor

Example:

In [36]:
tensor = rand_tensor()
print(tensor)

[[[49 48 20 30]
  [33 15 38  5]
  [46  0 48 35]
  [39 49 35  2]]

 [[28  3 46  6]
  [ 9 12 29  2]
  [20 27 27  4]
  [13 50 33 23]]

 [[41 28 22 44]
  [20 11 31 23]
  [25  7 29 23]
  [19 36  6  3]]]


In [76]:
A,B,C = CP(tensor, 5,100, 1e-4)
print(A)
print(B)
print(C)

[[13.68051212 17.12565491 39.39776144 33.7737343  -8.00895449]
 [ 9.69670908 -2.7811761  -3.56059073 31.68962907 12.85461262]
 [ 7.50726647  6.9039777  24.20482019 -8.61884572 -0.48499096]]
[[ 2.9388792  -0.90863981  1.34963287 -0.59638996 -0.08071789]
 [ 2.11041135 -0.47960289  0.2962909  -0.4589357  -0.51279064]
 [ 2.11168874 -0.04955963  0.30062702 -0.19628011 -0.98872989]
 [ 2.1456726  -2.55874086  1.18629673  0.21672995 -2.46211993]]
[[ 1.36355433  0.81689862  0.36149092  0.58751383  0.72628334]
 [ 0.29445706 -1.25343086  0.38691786  0.1269488  -1.69447927]
 [ 2.05795598  0.13682734 -0.82476922  0.85318895  0.64644747]
 [ 1.04814488  2.03959506  0.84077723  1.28485287  0.5963031 ]]


In [78]:
X = reconstruct_CP(A,B,C)
print(X)

[[[49.96741712 48.26438746 19.98936184 29.6146271 ]
  [30.75435431 14.38627098 38.09526573  5.85762438]
  [44.83639244 -0.10654794 49.03066601 34.71145146]
  [39.74626396 49.16828275 34.85810394  1.85097051]]

 [[27.3278288   2.72335167 46.160215    6.08171084]
  [15.28013451 13.26895532 26.49716007  0.66589654]
  [14.76114575 26.18926711 29.51824304  5.27271781]
  [13.70497959 50.07383614 32.6751957  22.72241262]]

 [[39.81685295 27.58554265 22.01384379 44.42424787]
  [23.99550357 11.67110354 29.77258873 21.11306434]
  [25.30953778  7.31457319 28.3297799  24.49594358]
  [17.68309615 35.73509883  6.22841026  3.30736798]]]


Definitions:
Rank of a tensor is given by the parameter sum(a_i o b_i o c_i) with the minimum i that best represents the tensor. 

A rank 1 tensor is given by a o b o c, where a,b,c are some columns vectors. 

Things to check:
Uniqueness of the decomposition? 
generalization into t_d tensors? (particularly for cp decomposition?)

Notice that we have 4 free parameters here, namely R, A, B, and C. In this particular example, I guessed the Rank of the tensor, and randomly generated A,B and C for this setup. 

If we don't know the rank of the tensor (which is usually the case), we can guess a max_rank and have the function iterate from i rank to max_rank, calculate the forbinus norm after performing ALS for each rank, and pick the rank that gives the best approximation. 

A seperate paper, https://arxiv.org/pdf/1410.6089, the authors offers a method to obtain a best rank 1 decomposition of a tensor by solving the least square problem using newton's method.

Newton1 Implementation:

Newton1 is a algorithm to find the best rank 1 approximation of a tensor using the newton's method (Newton-Raphson method).

In [82]:
def F(T,u,v,w): 
    x = np.einsum('ijk,j,k->i', T, v, w)
    y = np.einsum('ijk,i,k->j', T, u, w)
    z = np.einsum('ijk,i,j->k', T, u, v)
    return np.concatenate([x, y, z])

In [83]:
def G(T,u,v,w):
    return np.concatenate([u, v, w]) - F(T,u,v,w)

In [84]:
def DG(T,u,v,w):
    m, n, l = T.shape
    T_w = np.einsum('ijk,k->ij', T, w)
    T_v = np.einsum('ijk,j->ik', T, v)
    T_u = np.einsum('ijk,i->jk', T, u)

    Im = np.eye(m)
    In = np.eye(n)
    Il = np.eye(l)
    
    row1 = np.hstack([Im, -T_w, -T_v])
    row2 = np.hstack([-T_w.T, In, -T_u])
    row3 = np.hstack([-T_v.T, -T_u.T, Il])

    return np.vstack([row1, row2, row3])

In [204]:
def newton1(T,u,v,w):
    m, n, l = T.shape
    Gvector = G(T,u,v,w)
    DGmatrix = DG(T,u,v,w)
    hold = np.linalg.solve(DGmatrix, Gvector)

    long_vec = np.concatenate([u, v, w])-hold
    
    u = long_vec[:m]
    v = long_vec[m:m+n]
    w = long_vec[m+n:]
    return u,v,w

In [211]:
def initialize_newton1(T):
    T1, T2, T3 = flatten(T)
    
    u = np.linalg.svd(T1, full_matrices=False)[0][:, 0]
    v = np.linalg.svd(T2, full_matrices=False)[0][:, 0]
    w = np.linalg.svd(T3, full_matrices=False)[0][:, 0]

    u = np.abs(u)
    v = np.abs(v)
    w = np.abs(w)

    return u, v, w

Example

In [167]:
T = rand_tensor()
print(T)

[[[ 8  5  9 25]
  [28 19 46  8]
  [30  6 45 28]
  [18 45 43 13]]

 [[21 19 21 31]
  [44  8 31  0]
  [15 31 38 19]
  [43 46 20 10]]

 [[41 35 12 29]
  [45 22  6 19]
  [23 34 16  8]
  [11 21 45 21]]]


In [215]:
m,n,l = T.shape

u,v,w = initialize_newton1(T)

for i in range(50):
    u,v,w = newton1(T,u,v,w)
    if i%10==0:
        err = np.linalg.norm(G(T, u, v, w))
        print(np.linalg.norm(G(T, u, v, w)))
        if err < 1e-12: # Stop early if converged
            break

u /= np.linalg.norm(u)
v /= np.linalg.norm(v)
w /= np.linalg.norm(w)
sigma = np.einsum('ijk,i,j,k->', T, u, v, w)
T_approx = sigma * np.einsum('i,j,k->ijk', u, v, w)

print(sigma)
print(T_approx)

73.69308177060753
2.4472808301498805e-05
0.0
171.22872568749818
[[[21.93453498 20.1482729  23.09924249 13.61598915]
  [26.47841655 24.32211866 27.8844008  16.43662986]
  [27.08610328 24.88031778 28.52435525 16.8138549 ]
  [31.50450019 28.93889786 33.17736575 19.55660028]]

 [[22.82014364 20.96176107 24.03187631 14.16573584]
  [27.5474848  25.30412621 29.01023578 17.10026014]
  [28.17970693 25.88486266 29.67602843 17.49271568]
  [32.77649698 30.10730824 34.51690462 20.34619962]]

 [[21.23840854 19.50883625 22.36615225 13.18386465]
  [25.63808299 23.55021855 26.99944615 15.91498795]
  [26.22648384 24.09070236 27.61909065 16.28024117]
  [30.50465608 28.02047711 32.12443065 18.93594128]]]


Computation of the matrix DG is O(n^3), and thus the complexity of each iteration of newton1 is O(n^3)

Explain einstein notation used in the code!

Newton2 Implementation

In [216]:
diff = T - T_approx
frob_error = np.linalg.norm(diff)
    
# Relative Error (Standard in literature)
relative_error = frob_error / np.linalg.norm(T)
memory_bytes = u.nbytes + v.nbytes + w.nbytes + sigma.nbytes